In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize']=10,6

import warnings
warnings.filterwarnings("ignore")
import textwrap

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/ import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import nltk
import random
import string

In [ ]:
# from IPython.display import Markdown,display
# def printmd(string):
#     display(Markdown(string))

In [ ]:
df_food = pd.read_csv("../input/indian-food-101/indian_food.csv")

In [ ]:
df_food.head()

In [ ]:
df_food.info()

In [ ]:
# Check the cardinality of each columns
cardinality_df = pd.DataFrame()
l = []
for i in range(len(df_food.columns)):
#     print(i)
    distinct_count = df_food[df_food.columns[i]].value_counts().count()
    l.append(distinct_count)
col_name = df_food.columns.tolist()
cardinality_df["Columns_Name"]= col_name
cardinality_df['Cardinality_Count'] = l
cardinality_df

In [ ]:
def bar_chart(df,parameter, figsize=(10,6),palette='winter'):
    target_counts = df[parameter].value_counts()
    target_perc = target_counts.div(target_counts.sum(), axis=0)
    plt.figure(figsize=figsize)
    ax = sns.barplot(x=target_counts.index.values, y=target_counts.values, order=target_counts.index,palette='winter')
#     plt.xticks(rotation=90)
    plt.xlabel(f'{parameter}', weight ='bold',fontsize=16)
    plt.ylabel('# of occurances', weight ='bold',fontsize=16)
    plt.title("Count of "+f'{parameter}', weight ='bold',fontsize=20)

    rects = ax.patches
    labels = np.round(target_perc.values*100, 2)
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, f'{label}%', ha='center', va='bottom')
    
    try:
        labels =target_counts.index.tolist()

    #     labels.sort()
        labels=[textwrap.fill(text,12) for text in labels]
        pos = np.arange(len(labels)) 
        plt.xticks(pos, labels,fontsize=12)
    except:
        pass

In [ ]:
# Genersting the pie chart
def pie_chart(df,col,path):
  label = df[col].value_counts().index.tolist()
#   fig = plt.figure(figsize=(10,6))
  ax = (df[col].value_counts()*100.0 /len(df))\
  .plot.pie(startangle=90,autopct='%.1f%%', labels =label, fontsize=12)                                                                           
  ax.set_title('% '+str(col))
  # plt.savefig(path+str(col1)+'.png')
  plt.show()

## Distribution of Dishes per Region

In [ ]:
bar_chart(df_food,"region")

## Distribution of Course

In [ ]:
bar_chart(df_food,"course")

## Distribution of flavour profile

In [ ]:
bar_chart(df_food,"flavor_profile")

## Distribution of Diet

In [ ]:
pie_chart(df_food,'diet',8)

## Distribution of State

In [ ]:
fig=plt.figure(figsize=(10,10))
pie_chart(df_food,"state",8)

In [ ]:
df_food.columns

# Region VS Diet
### Distribution of Diet across different region

In [ ]:
temp = df_food.groupby(['region','diet'])[['diet']].count().add_prefix("count_of_").reset_index()
temp1 =temp.pivot('region','diet','count_of_diet')
ax = temp1.plot(kind="bar",figsize=(10,6),stacked=True)
plt.xticks(rotation=0)
# ax.legend(["Not_Fraud", "Fraud"])
plt.xlabel('Region',fontweight ="bold",fontsize=14)
plt.ylabel('Count',fontweight ="bold",fontsize=14)
plt.title('Region Vs Diet',fontweight ="bold",fontsize=16)
plt.show()

## Distribution of the diet across the state

In [ ]:
temp = df_food.groupby(['state','diet'])[['diet']].count().add_prefix("count_of_").reset_index()
temp1 =temp.pivot('state','diet','count_of_diet')
ax = temp1.plot(kind="bar",figsize=(20,6),stacked=True)
plt.xticks(rotation=0)
# ax.legend(["Not_Fraud", "Fraud"])
plt.xlabel('State',fontweight ="bold",fontsize=14)
plt.ylabel('Count',fontweight ="bold",fontsize=14)
plt.title('State Vs Diet',fontweight ="bold",fontsize=16)
# plt.show()
labels = df_food["state"].value_counts().index.tolist()
labels.sort()
labels=[textwrap.fill(text,9) for text in labels]
pos = np.arange(len(labels)) 
plt.xticks(pos, labels,fontsize=10)
#     plt.legend()
#     plt.savefig(path+str(x_axis)+'_'+'and'+'_'+str(y_axis)+'.jpg')    
plt.show()


In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


In [ ]:
def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

In [ ]:
# Applying the cleaning function to both Descriptions datasets
df_food['text_clean'] = df_food['ingredients'].apply(str).apply(lambda x: text_preprocessing(x))

In [ ]:
# Analyzing Text statistics

df_food['text_len'] = df_food['text_clean'].astype(str).apply(len)
df_food['text_word_count'] = df_food['text_clean'].apply(lambda x: len(str(x).split()))

## Distribution of ingredients

In [ ]:
plt.rcParams['figure.figsize']=10,6
sns.distplot(df_food['text_len'],color='red')
plt.xlabel('text_len',weight='bold',fontsize=12)
plt.ylabel('Count',weight='bold',fontsize=12)
plt.title('Distribution of ingredients',weight='bold',fontsize=14)
plt.show()

In [ ]:
#source of code : https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#Distribution of top unigrams
des_unigrams = get_top_n_words(df_food['text_clean'],20)

df1 = pd.DataFrame(des_unigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).plot(kind='barh',color='g')
plt.ylabel('words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 unigrams in description text',weight='bold',fontsize=14)
plt.show()

In [ ]:
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#Distribution of top Bigrams
des_bigrams = get_top_n_gram(df_food['text_clean'],(2,2),20)

df1 = pd.DataFrame(des_bigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).plot(kind='barh',color='r',alpha=0.5)
plt.ylabel('words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 bigrams in description text',weight='bold',fontsize=14)
plt.show()

In [ ]:
#Distribution of top Trigrams
des_trigrams = get_top_n_gram(df_food['text_clean'],(3,3),20)

df1 = pd.DataFrame(des_trigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).plot(kind='barh',color='b',alpha=0.5)
plt.ylabel('words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 trigrams in description text',weight='bold',fontsize=14)
plt.show()

In [ ]:
#source of code : https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
top_words_in_description_text = get_top_n_words(df_food['text_clean'])

p1 = [x[0] for x in top_words_in_description_text[:20]]
p2 = [x[1] for x in top_words_in_description_text[:20]]

In [ ]:
# Top ingredients word
sns.barplot(x=p2,y=p1,palette='winter')
# plt.xticks(rotation=45)
plt.yticks(fontsize=12)
plt.ylabel('Words',weight='bold',fontsize=12)
plt.xlabel('Counts',weight='bold',fontsize=12)
plt.title('Top 20 description Words',weight='bold',fontsize=14)
plt.show()

In [ ]:
from wordcloud import WordCloud
fig,ax = plt.subplots(figsize=[40,30])
wordcloud1 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(df_food['text_clean']))
ax.imshow(wordcloud1)
ax.axis('off')
ax.set_title('Ingredients text',fontsize=20);